# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value

print(sum)

generator = square_root_generator(13)

for sqrt_value in generator:
    print(sqrt_value)



8.382332347441762
1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [4]:
#Install the dependencies
# %%capture
!pip install 'dlt[duckdb]'

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [5]:
import dlt

# load the first generator

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
										table_name="http_download",
										write_disposition="replace")

print(info)

info = generators_pipeline.run(people_2(),
                               table_name="http_download",
                               write_disposition="append")

/Users/cris/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/cris/data-engineering-2024/dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708036227.0842292 is LOADED and contains no failed jobs


In [6]:
# let's see what we wrote

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people = conn.sql("SELECT * FROM http_download").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708036227.0842292,9X3JsYxNpxKRrg,None
1,2,Person_2,27,City_A,1708036227.0842292,e41WTjq8LWBEvQ,None
2,3,Person_3,28,City_A,1708036227.0842292,VVhIa9vBumOxVQ,None
3,4,Person_4,29,City_A,1708036227.0842292,C4D+Yj10UW/gig,None
4,5,Person_5,30,City_A,1708036227.0842292,BFk8SrxM6GENfA,None
5,3,Person_3,33,City_B,1708036227.699143,APqp5lqpoFU1KA,Job_3
6,4,Person_4,34,City_B,1708036227.699143,kt8IchGpd5L4sw,Job_4
7,5,Person_5,35,City_B,1708036227.699143,ndlkajbvVFt7og,Job_5
8,6,Person_6,36,City_B,1708036227.699143,ZMxY+qtJWuT39Q,Job_6
9,7,Person_7,37,City_B,1708036227.699143,oWmYI5Ro95e5WQ,Job_7


In [7]:
AGE = conn.sql("SELECT sum(age) FROM http_download").df()
display(AGE)

,sum(age)
0,353.0


In [8]:
generators_pipeline.run(people_1(),
										table_name="http_overwrite",
										write_disposition="replace")

generators_pipeline.run(people_2(),
                        table_name="http_overwrite",
                        write_disposition="merge",
                        primary_key="id")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x11d484d00>, metrics={'1708036307.537811': [{'started_at': DateTime(2024, 2, 15, 22, 31, 47, 921028, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 15, 22, 31, 48, 256286, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////Users/cris/data-engineering-2024/dlt-workshop/dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='generators', loads_ids=['1708036307.537811'], load_packages=[LoadPackageInfo(load_id='1708036307.537811', package_path='/Users/cris/.dlt/pipelines/dlt_ipykernel_launcher/load/loaded/1708036307.537811', state='loaded', schema=Schema dlt_ipykernel_launcher at 4392064768, schema_update={'http_overwrite': {'name': 'http_overwrite', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'te

In [9]:
people = conn.sql("SELECT * FROM http_overwrite").df()
display(people)

AGE = conn.sql("SELECT sum(age) FROM http_overwrite").df()
display(AGE)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708036306.948889,xKLAJ57XnI8UrQ,None
1,2,Person_2,27,City_A,1708036306.948889,vKLP2/SUwR4b9A,None
2,4,Person_4,34,City_B,1708036307.537811,j85WvuQ2PzLnsA,Job_4
3,5,Person_5,35,City_B,1708036307.537811,vF0AkXWFfv1mWQ,Job_5
4,3,Person_3,33,City_B,1708036307.537811,yC9mdu2OeK1haw,Job_3
5,6,Person_6,36,City_B,1708036307.537811,D4XTfsngwNzERg,Job_6
6,7,Person_7,37,City_B,1708036307.537811,9P+zQazUeYPMnA,Job_7
7,8,Person_8,38,City_B,1708036307.537811,nTQYmAFF4A08gw,Job_8


,sum(age)
0,266.0
